# Imports

In [1]:
import API_functions.file_single as fs
import API_functions.file_batch as fb
import API_functions.file_info as fi
import API_functions.pre_process as pp
import API_functions.threshold_position_independent as tpi

# A quick check about the method

In [4]:
# Read image
single_path = "e:/3.Experimental_Data/Soil.column.2/0201/Reconstruct.Rename/"
file_name = "0103_rec00003312.bmp"
image = fs.read_image_opencv(str(os.path.join(single_path, file_name)))
fs.show_image(image, 4, (2, 2, 1))

# Get image info
fi.get_info(image)
hist = fi.calculate_hist(image)
fi.plot_hist(hist, 4, (2, 2, 2))

# Pre-process image
image_prepropossed = pp.median(image, kernel_size=3)
fs.show_image(image_prepropossed, 4, (2, 2, 3))

# Threshold image
image_threshold = tpi.user_threshold(image_prepropossed, 145)
# image_threshold = tpi.kmeans_3d(image_prepropossed)
fs.show_image(image_threshold, 4, (2, 2, 4))

Exception: Error: Image not found

# Work with the method above

## 2.1 Get Image Lists

In [2]:
name_to_read = fb.ImageName(prefix="1-2_rec00", suffix="")
path_to_read = "e:/3.Experimental_Data/Soil.column.2/0202/Reconstruct/"
# path_to_read = "e:/3.Experimental_Data/Soil.column.2/0201/Threshold/"
list_to_read = fb.get_image_names(path_to_read, name_to_read, "bmp")

Your image name format is: 1-2_rec00_XXXX
3506 images have been found in e:/3.Experimental_Data/Soil.column.2/0202/Reconstruct/
The first 3 images are:
e:/3.Experimental_Data/Soil.column.2/0202/Reconstruct\1-2_rec00000014.bmp
e:/3.Experimental_Data/Soil.column.2/0202/Reconstruct\1-2_rec00000015.bmp
e:/3.Experimental_Data/Soil.column.2/0202/Reconstruct\1-2_rec00000016.bmp
Warning, your files are too large to read all.
Get names completely!


## 2.2 Rename and ROI select

In [3]:
roi = fb.roi_region(100, 300, 800, 1000, z1=16, z2=50)

Your ROI is: x1=100, y1=300, width=800, height=1000, depth=16-50


In [14]:
path_roi_selected = "e:/3.Experimental_Data/Soil.column.2/0202/ROI.Rename/"
list_roi_selected = fb.roi_select(list_to_read, path=path_roi_selected, roi=roi)

100%|██████████| 35/35 [00:00<00:00, 43.77it/s]

The first 3 images are:
e:/3.Experimental_Data/Soil.column.2/0202/ROI.Rename/1-2_rec00000016.png
e:/3.Experimental_Data/Soil.column.2/0202/ROI.Rename/1-2_rec00000017.png
e:/3.Experimental_Data/Soil.column.2/0202/ROI.Rename/1-2_rec00000018.png
ROI Selected Completely!


In [15]:
name_roi_selected = fb.ImageName(prefix="002_ou_DongYing", suffix="roi_selected")
startid = 3321
list_roi_selected.reverse()
list_renamed = fb.rename(list_roi_selected, new_name=name_roi_selected, start_index=startid, overwrite=True)

Your image name format is: 002_ou_DongYing_XXXX_roi_selected


100%|██████████| 35/35 [00:00<00:00, 696.09it/s]

The first 3 images are:
e:/3.Experimental_Data/Soil.column.2/0202/ROI.Rename\002_ou_DongYing_03321_roi_selected.png
e:/3.Experimental_Data/Soil.column.2/0202/ROI.Rename\002_ou_DongYing_03322_roi_selected.png
e:/3.Experimental_Data/Soil.column.2/0202/ROI.Rename\002_ou_DongYing_03323_roi_selected.png
Rename completely!


## 3.3 Pre-process and Get its Threshold

In [16]:
import cv2
import os
from tqdm import tqdm

In [17]:
def image_process(namelists: list, save_path: str, save_name: fb.ImageName):
    """
    Only for gray scale image.
    """
    
    temp_list = []

    for i, name in enumerate(tqdm(namelists)):
        image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        # image = fs.read_image_opencv(str(os.path.join(batch_read_path, name)))
        image_prepropossed = pp.median(image, 5)
        image_threshold = tpi.user_threshold(image_prepropossed, 145)
        image_invert = cv2.bitwise_not(image_threshold)     # invert image, make the pore space to be white

        save = os.path.join(save_path, f'{save_name.prefix}{str(i).zfill(5)}{save_name.suffix}.png')
        
        cv2.imwrite(save, image_invert)
        temp_list.append(save)

    fb.show_image_names(temp_list)
    return temp_list

In [18]:
name_processed = fb.ImageName(prefix="002_ou_DongYing", suffix="processed")
path_processed = "e:/3.Experimental_Data/Soil.column.2/0202/Threshold/"
list_processed = image_process(list_renamed, save_name=name_processed, save_path=path_processed)

Your image name format is: 002_ou_DongYing_XXXX_processed


100%|██████████| 35/35 [00:00<00:00, 64.09it/s]

The first 3 images are:
e:/3.Experimental_Data/Soil.column.2/0202/Threshold/002_ou_DongYing_00000_processed.png
e:/3.Experimental_Data/Soil.column.2/0202/Threshold/002_ou_DongYing_00001_processed.png
e:/3.Experimental_Data/Soil.column.2/0202/Threshold/002_ou_DongYing_00002_processed.png


## 3.4 Get nii File

In [19]:
name_nii = "soil_column_2_2'"
path_nii = "e:/3.Experimental_Data/Soil.column.2/"
fb.create_nifti(list_processed, path_nii, name_nii)

100%|██████████| 35/35 [00:00<00:00, 343.02it/s]

35 images have been read
Reading completely!


Exception: Error: The file has existed, please change the name.